- https://github.com/langchain-ai/langchain/tree/master/cookbook
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel
    - https://python.langchain.com/v0.1/docs/expression_language/why/
- https://ai.plainenglish.io/understanding-large-language-model-based-agents-27bee5c82cec

In [1]:
import os
from dotenv import load_dotenv
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_API_KEY=
# OPENAI_API_KEY=
load_dotenv()

True

## LangChain

- 所谓的 `agent` 开发，LLMs workflows，GenAI 时代的软件工程
    - 丰富的生态，
    - workflows 的复杂，手撸的效率非常低，而且不好维护，
    - Input -> Processing -> Output
- 与 AutoGen 等相比，更多地面向开发者，面向软件工程
    - LangGraph：multi-agents workflows
    - `LangSmith` 也在更多地弥补中间过程显示的不足
- 推荐 《大模型应用开发 动手做AI Agent》（https://www.bilibili.com/opus/935785456083140628?spm_id_from=333.999.0.0）
    - 面向开发者，第一本
    - 系统而全面，可以做一个很好的入门

## LCEL (LangChain Expression Language)

- LangChain 重写了 `|`（`__or__`），`Chain 之所在`
- `RunnablePassthrough`: RunnablePassthrough 允许你将输入数据直接传递而不做任何更改（identity），通常与 RunnableParallel 一起使用，将数据传递到新的键中。
- LangChain 的应用 RunnablePassthrough 作为一个占位符，可以在需要时填充数据，比如在公司名称尚未确定时先留空，后续再填入。
- 所谓的最佳实践
    - python：遍历，也可以用 list comprehension
    - 对于 matlab：也可以遍历，也可以整理成 matrix，直接矩阵矢量乘法；

In [35]:
# (2 | 3) > 3
2 | 3 > 2

True

In [32]:
2 | 3

3

In [4]:
from langchain_core.runnables import (
    RunnablePassthrough, 
    RunnableLambda, 
    RunnableParallel
)

In [2]:
os.environ["LANGCHAIN_PROJECT"] = 'lcel_test'

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-3.5-turbo")

# lcel
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream truck break down?\n\nBecause it had too many "scoops"!'

In [6]:
prompt.invoke({'topic': 'ice cream'})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about ice cream')])

In [7]:
llm.invoke(prompt.invoke({'topic': 'ice cream'}))

AIMessage(content='Why did the ice cream truck break down?\n\nIt had too many "scoops" of ice cream!', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb46f4ce-665b-45f7-9fd0-9dd559465fcc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37})

In [8]:
output_parser.invoke(llm.invoke(prompt.invoke({'topic': 'ice cream'})))

'Why did the ice cream truck break down? It had too many "scoops" on board!'

### runnables

- `RunnablePassthrough()`: identity

In [57]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [59]:
chain = RunnablePassthrough() | RunnableLambda(lambda x: x.upper())
chain.invoke("hello")

'HELLO'

In [60]:
chain = RunnablePassthrough() | RunnableLambda(lambda x: x.upper()) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

### json test

In [9]:
os.environ["LANGCHAIN_PROJECT"] = 'json_test2'

In [10]:
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [11]:
llm = ChatOpenAI(model="gpt-4o", 
                 model_kwargs={'response_format': {"type": "json_object"}})

In [12]:
json_parser = JsonOutputParser()

In [13]:
# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", '''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''),
    ("human", "{input}")
])

# 创建 LCEL 链
chain = (
    {"input": RunnablePassthrough()} 
    | prompt 
    | llm 
    | json_parser
)

In [14]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [15]:
print(prompt[0])
print(prompt[1])

prompt=PromptTemplate(input_variables=[], template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}')
prompt=PromptTemplate(input_variables=['input'], template='{input}')


In [16]:
ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template('''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''), 
     HumanMessagePromptTemplate.from_template("{input}")
    ]
)

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [17]:
result = chain.invoke("John is 20 years old. He is a student at the University of California, Berkeley. He is a very smart student.")

In [18]:
result

{'name': 'John',
 'age': 20,
 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}

In [19]:
# chain.invoke({'input': "John is 20 years old. He is a student at the University of California, Berkeley. He is a very smart student."})

### RAG

In [6]:
# !conda install faiss-gpu -c pytorch

In [21]:
os.environ["LANGCHAIN_PROJECT"] = 'rag_test'

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [22]:
OpenAIEmbeddings()

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7677f6861ac0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7677f6643e60>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [23]:
vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=OpenAIEmbeddings()
)

In [24]:
retriever = vectorstore.as_retriever()

In [25]:
retriever.invoke("What do cats like to eat?")

[Document(page_content='Cats love thuna')]

In [26]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [27]:
rag_chain.invoke("What do cats like to eat?")

'Tuna'

## Tool uses 

- precise math calculation
- custom tools (自定义 functions)

In [28]:
os.environ["LANGCHAIN_PROJECT"] = 'tools_test2'

In [30]:
import numpy as np
from langchain_core.tools import Tool
from langchain_core.tools import tool

In [31]:
@tool
def add(num1: float, num2: float) -> float:
    "Add two numbers."
    return num1 + num2
    
@tool
def subtract(num1: float, num2: float) -> float:
    """
    Subtract two numbers.
    """
    return num1 - num2
    
@tool
def multiply(num1: float, num2: float) -> float:
    """Multiply two float ."""
    return num1 * num2

@tool
def divide(numerator: float, denominator: float) -> float:
    """
    Divides the numerator by the denominator.
    """

    result = numerator / denominator
    return result

@tool
def power(base: float, exponent: float) -> float:
    "Take the base to the exponent power, base^exponent."
    return base**exponent


@tool
def exp(x):
    """
    Calculate the natural exponential $e^x$
    """
    return np.exp(x)

In [32]:
tools = [add, subtract, multiply, divide, power, exp]

In [33]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [34]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

system_template = """
You are a helpful math assistant that uses calculation functions to solve complex math problems step by step.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, tools, prompt)

In [35]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
#  """Agent that is using tools."""
# AgentExecutor??

In [36]:
agent_executor.invoke({"input": "What is the result of directive of sigmoid(2.5)?"})



> Entering new AgentExecutor chain...

Invoking: `exp` with `{'x': -2.5}`
responded: The sigmoid function is defined as:

\[ \sigma(x) = \frac{1}{1 + e^{-x}} \]

The derivative of the sigmoid function is:

\[ \sigma'(x) = \sigma(x) \cdot (1 - \sigma(x)) \]

First, we need to calculate \(\sigma(2.5)\):

\[ \sigma(2.5) = \frac{1}{1 + e^{-2.5}} \]

Let's calculate \(e^{-2.5}\) and then \(\sigma(2.5)\).

0.0820849986238988
Invoking: `divide` with `{'numerator': 1, 'denominator': 1.082085}`
responded: We have \( e^{-2.5} \approx 0.082085 \).

Now, we can calculate \(\sigma(2.5)\):

\[ \sigma(2.5) = \frac{1}{1 + 0.082085} \]

Let's compute this value.

0.9241418188035136
Invoking: `subtract` with `{'num1': 1, 'num2': 0.9241418188035136}`
responded: We have \(\sigma(2.5) \approx 0.9241\).

Next, we need to calculate the derivative \(\sigma'(2.5)\):

\[ \sigma'(2.5) = \sigma(2.5) \cdot (1 - \sigma(2.5)) \]

Let's compute \(1 - \sigma(2.5)\) and then \(\sigma'(2.5)\).

0.07585818119648635
Inv

{'input': 'What is the result of directive of sigmoid(2.5)?',
 'output': 'The derivative of the sigmoid function at \\(x = 2.5\\) is approximately \\(0.0701\\).'}

In [1]:
import torch
import torch.nn.functional as F

$$
\begin{split}
\sigma(x)&=\frac{1}{1+\exp(-x)}\\
\sigma'(x)&=\sigma(x)(1-\sigma(x))\\
\end{split}
$$

In [2]:
F.sigmoid(torch.tensor([2.5])) * (1-F.sigmoid(torch.tensor([2.5])))

tensor([0.0701])